In [1]:
import pandas as pd
import os

## Create trade data DataFrame

In [2]:
# Create empty trade data list and point to csv file directory
trade_data = []
files = os.listdir("stockdata")

# Import each csv returned by API, extract symbol from file name, and insert as column
for file in files:
    if file.endswith(".csv"):
        trade = pd.read_csv(os.path.join("stockdata", file))
        trade["Symbol"] = file.split("_")[0]
        trade_data.append(trade)

# Combine all individual DataFrames into one DataFrame
trade_df = pd.concat(trade_data, ignore_index=True)

# Preview result
trade_df

,timestamp,open,high,low,close,volume,Symbol
0,2023-09-12,189.485,189.9800,173.5400,176.30,500216459,AAPL
1,2023-08-31,196.235,196.7300,171.9600,187.87,1323817340,AAPL
2,2023-07-31,193.780,198.2300,186.6000,196.45,996368613,AAPL
3,2023-06-30,177.700,194.4800,176.9306,193.97,1297863403,AAPL
4,2023-05-31,169.280,179.3500,164.3100,177.25,1275052503,AAPL
...,...,...,...,...,...,...,...
22696,2018-08-31,35.600,48.2400,35.1913,42.79,26382167,ZS
22697,2018-07-31,36.000,42.6300,33.9000,35.31,21627971,ZS
22698,2018-06-29,26.360,43.9799,26.3600,35.75,51552171,ZS
22699,2018-05-31,29.800,30.1000,24.7600,26.26,12958084,ZS


## Create Sector DataFrame and csv

In [3]:
# Import metadata csv returned by API and convert into a DataFrame
all_metadata = pd.read_csv("stockdata/Metadata/overview_nasdaq.csv", encoding='UTF-8')
all_metadata.head()

,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,Industry,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,TECHNOLOGY,ELECTRONIC COMPUTERS,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
1,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,USA,TECHNOLOGY,SERVICES-PREPACKAGED SOFTWARE,...,9.21,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16
2,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,USA,TRADE & SERVICES,RETAIL-CATALOG & MAIL-ORDER HOUSES,...,1.948,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None
3,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,USA,MANUFACTURING,SEMICONDUCTORS & RELATED DEVICES,...,14.59,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06
4,META,Common Stock,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,USA,TECHNOLOGY,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING...",...,2.595,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None


In [4]:
# Exctract the sector column from the metadata Dataframe
sector_list = all_metadata[["Sector"]]
sector_list.head()

,Sector
0,TECHNOLOGY
1,TECHNOLOGY
2,TRADE & SERVICES
3,MANUFACTURING
4,TECHNOLOGY


In [5]:
# Keep only unique instances of sector
sector_base = sector_list.drop_duplicates().reset_index(drop=True)
sector_base

,Sector
0,TECHNOLOGY
1,TRADE & SERVICES
2,MANUFACTURING
3,LIFE SCIENCES
4,ENERGY & TRANSPORTATION
5,REAL ESTATE & CONSTRUCTION


In [6]:
# Create a unique id for each sector
sector_base["sector_id"] = range(len(sector_base))
id_column = sector_base.pop("sector_id")
sector_base.insert(0, "sector_id", id_column)


sector_base

,sector_id,Sector
0,0,TECHNOLOGY
1,1,TRADE & SERVICES
2,2,MANUFACTURING
3,3,LIFE SCIENCES
4,4,ENERGY & TRANSPORTATION
5,5,REAL ESTATE & CONSTRUCTION


In [7]:
# Rename column
sector = sector_base.rename(columns={"Sector": "sector"})

In [8]:
# Export final DataFrame to csv (for later import into DB)
sector.to_csv("clean_db_data/clean_sector.csv", encoding='UTF-8', index=False)

## Create Industry DataFrame and csv

In [9]:
# Exctract the industry column from the metadata Dataframe
industry_list = all_metadata[["Industry"]]
industry_list.head()

,Industry
0,ELECTRONIC COMPUTERS
1,SERVICES-PREPACKAGED SOFTWARE
2,RETAIL-CATALOG & MAIL-ORDER HOUSES
3,SEMICONDUCTORS & RELATED DEVICES
4,"SERVICES-COMPUTER PROGRAMMING, DATA PROCESSING..."


In [14]:
# Keep only unique instances of sector
industry_base = industry_list.drop_duplicates().reset_index(drop=True)

In [15]:
# Create a unique id for each industry
industry_base["industry_id"] = range(len(industry_base))
id_column = industry_base.pop("industry_id")
industry_base.insert(0, "industry_id", id_column)

In [12]:
# Rename column
industry = industry_base.rename(columns={"Industry": "industry"})

In [13]:
# Export final DataFrame to csv (for later import into DB)
industry.to_csv("clean_db_data/clean_industry.csv", encoding='UTF-8', index=False)

## Create Asset Type DataFrame and csv

In [23]:
# Exctract the asset type column from the metadata Dataframe
asset_list = all_metadata[["AssetType"]]
asset_list.head()

,AssetType
0,Common Stock
1,Common Stock
2,Common Stock
3,Common Stock
4,Common Stock


In [24]:
# Keep only unique instances of asset type
asset_base = asset_list.drop_duplicates().reset_index(drop=True)
asset_base

,AssetType
0,Common Stock


In [25]:
# Create a unique id for each asset type
asset_base["asset_type_id"] = range(len(asset_base))
id_column = asset_base.pop("asset_type_id")
asset_base.insert(0, "asset_type_id", id_column)
asset_base

,asset_type_id,AssetType
0,0,Common Stock


In [26]:
# Rename column
asset_type = asset_base.rename(columns={"AssetType": "asset_type"})
asset_type                                        

,asset_type_id,asset_type
0,0,Common Stock


In [27]:
# Export final DataFrame to csv (for later import into DB)
asset_type.to_csv("clean_db_data/clean_asset_type.csv", encoding='UTF-8', index=False)

## Create Country DataFrame and csv

In [28]:
# Exctract the country column from the metadata Dataframe
country_list = all_metadata[["Country"]]
country_list.head()

,Country
0,USA
1,USA
2,USA
3,USA
4,USA


In [29]:
# Keep only unique instances of country
country_base = country_list.drop_duplicates().reset_index(drop=True)
country_base

,Country
0,USA
1,China


In [30]:
# Create a unique id for each country
country_base["country_id"] = range(len(country_base))
id_column = country_base.pop("country_id")
country_base.insert(0, "country_id", id_column)
country_base

,country_id,Country
0,0,USA
1,1,China


In [31]:
# Rename column
country = country_base.rename(columns={"Country": "country"})
country

,country_id,country
0,0,USA
1,1,China


In [32]:
# Export final DataFrame to csv (for later import into DB)
country.to_csv("clean_db_data/clean_country.csv", encoding='UTF-8', index=False)

## Create Currency DataFrame and csv

In [33]:
# Exctract the country column from the metadata Dataframe
currency_list = all_metadata[["Currency"]]
currency_list.head()

,Currency
0,USD
1,USD
2,USD
3,USD
4,USD


In [35]:
# Keep only unique instances of country
currency_base = currency_list.drop_duplicates().reset_index(drop=True)
currency_base

,Currency
0,USD


In [36]:
# Create a unique id for each country
currency_base["currency_id"] = range(len(currency_base))
id_column = currency_base.pop("currency_id")
currency_base.insert(0, "currency_id", id_column)
currency_base

,currency_id,Currency
0,0,USD


In [37]:
# Rename column
currency = currency_base.rename(columns={"Currency": "currency"})
currency

,currency_id,currency
0,0,USD


In [38]:
# Export final DataFrame to csv (for later import into DB)
currency.to_csv("clean_db_data/clean_currency.csv", encoding='UTF-8', index=False)

## Create Metadata DataFrame

In [40]:
# Create a unique ID for each symbol
all_metadata["ID"] = range(len(all_metadata))

In [41]:
# Move ID column to start of DataFrame and preview
id_column = all_metadata.pop("ID")
all_metadata.insert(0, "ID", id_column)
all_metadata.head()

,ID,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,TECHNOLOGY,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
1,1,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,USA,TECHNOLOGY,...,9.21,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16
2,2,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,USA,TRADE & SERVICES,...,1.948,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None
3,3,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,USA,MANUFACTURING,...,14.59,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06
4,4,META,Common Stock,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,USA,TECHNOLOGY,...,2.595,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None


In [42]:
# Merge overview DataFrame with sector DataFrame
merged_sector = pd.merge(all_metadata, sector_base, how="left", on="Sector")
merged_sector.head()

,ID,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,Sector,...,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate,sector_id
0,0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,TECHNOLOGY,...,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11,0
1,1,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,USA,TECHNOLOGY,...,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16,0
2,2,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,USA,TRADE & SERVICES,...,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None,1
3,3,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,USA,MANUFACTURING,...,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06,2
4,4,META,Common Stock,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,USA,TECHNOLOGY,...,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None,0


In [43]:
# Remove "Sector" column to leave only sector_id
overview = merged_sector.drop(columns="Sector")
sector_id = overview.pop("sector_id")
overview.insert(1, "sector_id", sector_id)
overview

,ID,sector_id,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,0,0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,...,5.92,23.52,1.275,197.96,123.640,186.290,164.040,15634200000,2023-08-17,2023-08-11
1,1,0,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,USA,...,9.21,18.55,0.905,366.01,211.390,332.700,290.880,7429760000,2023-09-14,2023-08-16
2,2,1,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,USA,...,1.948,19.3,1.236,143.63,81.430,133.560,109.960,10317800000,None,None
3,3,2,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,USA,...,14.59,56.0,1.736,502.62,108.060,451.030,301.260,2470000000,2023-09-28,2023-09-06
4,4,0,META,Common Stock,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,USA,...,2.595,7.05,1.234,326.20,88.090,300.410,217.840,2222580000,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,0,ZM,Common Stock,Zoom Video Communications Inc,"Zoom Video Communications, Inc. provides a pre...",1585521,NASDAQ,USD,USA,...,3.759,23.41,-0.218,89.67,60.450,68.850,69.940,254292000,None,None
97,97,0,SIRI,Common Stock,Sirius XM Holding Inc,Sirius XM Holdings Inc. provides satellite rad...,908937,NASDAQ,USD,USA,...,3.765,13.43,1.053,7.91,3.282,4.781,4.664,3842400000,2023-08-30,2023-08-07
98,98,2,ENPH,Common Stock,Enphase Energy Inc,Enphase Energy is an American energy technolog...,1463101,NASDAQ,USD,USA,...,22.34,110.55,1.512,339.92,118.140,149.690,201.510,136355000,None,None
99,99,1,JD,Common Stock,JD.com Inc Adr,"JD.com, Inc. is an e-commerce company and reta...",1549802,NASDAQ,USD,China,...,0.0855,12.96,0.539,66.10,31.570,36.110,43.870,1573000000,2023-05-04,2023-04-05


In [45]:
# Merge with industry dataframe
merged_industry = pd.merge(overview, industry_base, how="left", on="Industry")
merged_industry.head()

,ID,sector_id,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,Country,...,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate,industry_id
0,0,0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,USA,...,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11,0
1,1,0,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,USA,...,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16,1
2,2,1,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,USA,...,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None,2
3,3,2,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,USA,...,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06,3
4,4,0,META,Common Stock,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,USA,...,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None,4


In [47]:
# Remove "Industry" column to leave only sector_id
overview_1 = merged_industry.drop(columns="Industry")
industry_id = overview_1.pop("industry_id")
overview_1.insert(2, "industry_id", industry_id)
overview_1

,ID,sector_id,industry_id,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,0,0,0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,...,5.92,23.52,1.275,197.96,123.640,186.290,164.040,15634200000,2023-08-17,2023-08-11
1,1,0,1,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,...,9.21,18.55,0.905,366.01,211.390,332.700,290.880,7429760000,2023-09-14,2023-08-16
2,2,1,2,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,...,1.948,19.3,1.236,143.63,81.430,133.560,109.960,10317800000,None,None
3,3,2,3,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,...,14.59,56.0,1.736,502.62,108.060,451.030,301.260,2470000000,2023-09-28,2023-09-06
4,4,0,4,META,Common Stock,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,...,2.595,7.05,1.234,326.20,88.090,300.410,217.840,2222580000,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
96,96,0,4,ZM,Common Stock,Zoom Video Communications Inc,"Zoom Video Communications, Inc. provides a pre...",1585521,NASDAQ,USD,...,3.759,23.41,-0.218,89.67,60.450,68.850,69.940,254292000,None,None
97,97,0,48,SIRI,Common Stock,Sirius XM Holding Inc,Sirius XM Holdings Inc. provides satellite rad...,908937,NASDAQ,USD,...,3.765,13.43,1.053,7.91,3.282,4.781,4.664,3842400000,2023-08-30,2023-08-07
98,98,2,3,ENPH,Common Stock,Enphase Energy Inc,Enphase Energy is an American energy technolog...,1463101,NASDAQ,USD,...,22.34,110.55,1.512,339.92,118.140,149.690,201.510,136355000,None,None
99,99,1,49,JD,Common Stock,JD.com Inc Adr,"JD.com, Inc. is an e-commerce company and reta...",1549802,NASDAQ,USD,...,0.0855,12.96,0.539,66.10,31.570,36.110,43.870,1573000000,2023-05-04,2023-04-05


In [48]:
# Merge with asset type dataframe
merged_asset = pd.merge(overview_1, asset_base, how="left", on="AssetType")
merged_asset.head()

,ID,sector_id,industry_id,Symbol,AssetType,Name,Description,CIK,Exchange,Currency,...,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate,asset_type_id
0,0,0,0,AAPL,Common Stock,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,...,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11,0
1,1,0,1,MSFT,Common Stock,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,...,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16,0
2,2,1,2,AMZN,Common Stock,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,...,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None,0
3,3,2,3,NVDA,Common Stock,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,...,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06,0
4,4,0,4,META,Common Stock,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,...,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None,0


In [62]:
# Remove "AssetType" column to leave only asset_type_id
overview_2 = merged_asset.drop(columns="AssetType")
asset_type_id = overview_2.pop("asset_type_id")
overview_2.insert(3, "asset_type_id", asset_type_id)
overview_2.head()

,ID,sector_id,industry_id,asset_type_id,Symbol,Name,Description,CIK,Exchange,Currency,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,0,0,0,0,AAPL,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
1,1,0,1,0,MSFT,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,...,9.21,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16
2,2,1,2,0,AMZN,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,...,1.948,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None
3,3,2,3,0,NVDA,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,...,14.59,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06
4,4,0,4,0,META,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,...,2.595,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None


In [54]:
# Merge with country dataframe
merged_country = pd.merge(overview_2, country_base, how="left", on="Country")
merged_country.head()

,ID,sector_id,industry_id,asset_type_id,Symbol,Name,Description,CIK,Exchange,Currency,...,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate,country_id
0,0,0,0,0,AAPL,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,USD,...,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11,0
1,1,0,1,0,MSFT,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,USD,...,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16,0
2,2,1,2,0,AMZN,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,USD,...,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None,0
3,3,2,3,0,NVDA,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,USD,...,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06,0
4,4,0,4,0,META,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,USD,...,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None,0


In [61]:
# Remove "Country" column to leave only country_id
overview_3 = merged_country.drop(columns="Country")
country_id = overview_3.pop("country_id")
overview_3.insert(4, "country_id", country_id)
overview_3.head()

,ID,sector_id,industry_id,asset_type_id,country_id,Symbol,Name,Description,CIK,Exchange,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,0,0,0,0,0,AAPL,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
1,1,0,1,0,0,MSFT,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,...,9.21,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16
2,2,1,2,0,0,AMZN,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,...,1.948,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None
3,3,2,3,0,0,NVDA,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,...,14.59,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06
4,4,0,4,0,0,META,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,...,2.595,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None


In [58]:
# Merge with currency dataframe
merged_currency = pd.merge(overview_3, currency_base, how="left", on="Currency")
merged_currency.head()

,ID,sector_id,industry_id,asset_type_id,country_id,Symbol,Name,Description,CIK,Exchange,...,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate,currency_id
0,0,0,0,0,0,AAPL,Apple Inc,Apple Inc. is an American multinational techno...,320193,NASDAQ,...,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11,0
1,1,0,1,0,0,MSFT,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,NASDAQ,...,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16,0
2,2,1,2,0,0,AMZN,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,NASDAQ,...,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None,0
3,3,2,3,0,0,NVDA,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,NASDAQ,...,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06,0
4,4,0,4,0,0,META,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,NASDAQ,...,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None,0


In [60]:
# Remove "Currency" column to leave only currency_id
overview_4 = merged_currency.drop(columns="Currency")
currency_id = overview_4.pop("currency_id")
overview_4.insert(4, "currency_id", currency_id)
overview_4.head()

,ID,sector_id,industry_id,asset_type_id,currency_id,country_id,Symbol,Name,Description,CIK,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,0,0,0,0,0,0,AAPL,Apple Inc,Apple Inc. is an American multinational techno...,320193,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
1,1,0,1,0,0,0,MSFT,Microsoft Corporation,Microsoft Corporation is an American multinati...,789019,...,9.21,18.55,0.905,366.01,211.39,332.70,290.88,7429760000,2023-09-14,2023-08-16
2,2,1,2,0,0,0,AMZN,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1018724,...,1.948,19.3,1.236,143.63,81.43,133.56,109.96,10317800000,None,None
3,3,2,3,0,0,0,NVDA,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1045810,...,14.59,56.0,1.736,502.62,108.06,451.03,301.26,2470000000,2023-09-28,2023-09-06
4,4,0,4,0,0,0,META,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",1326801,...,2.595,7.05,1.234,326.20,88.09,300.41,217.84,2222580000,None,None


## Create trade data file

In [67]:
# Merge trade data and metadata dataframes on ticker symbol
combined_df = trade_df.merge(overview, how="left", on="Symbol")
combined_df.head()

,timestamp,open,high,low,close,volume,Symbol,ID,sector_id,AssetType,...,EVToRevenue,EVToEBITDA,Beta,52WeekHigh,52WeekLow,50DayMovingAverage,200DayMovingAverage,SharesOutstanding,DividendDate,ExDividendDate
0,2023-09-12,189.485,189.98,173.5400,176.30,500216459,AAPL,0,0,Common Stock,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
1,2023-08-31,196.235,196.73,171.9600,187.87,1323817340,AAPL,0,0,Common Stock,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
2,2023-07-31,193.780,198.23,186.6000,196.45,996368613,AAPL,0,0,Common Stock,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
3,2023-06-30,177.700,194.48,176.9306,193.97,1297863403,AAPL,0,0,Common Stock,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11
4,2023-05-31,169.280,179.35,164.3100,177.25,1275052503,AAPL,0,0,Common Stock,...,5.92,23.52,1.275,197.96,123.64,186.29,164.04,15634200000,2023-08-17,2023-08-11


In [69]:
# Edit DataFrame to exclude all columns from Metadata DataFrame except sector and ticker ID 
reduced_trade_df = combined_df[["ID", "timestamp", "open", "high", "low", "close", "volume"]]
reduced_trade_df

,ID,timestamp,open,high,low,close,volume
0,0,2023-09-12,189.485,189.9800,173.5400,176.30,500216459
1,0,2023-08-31,196.235,196.7300,171.9600,187.87,1323817340
2,0,2023-07-31,193.780,198.2300,186.6000,196.45,996368613
3,0,2023-06-30,177.700,194.4800,176.9306,193.97,1297863403
4,0,2023-05-31,169.280,179.3500,164.3100,177.25,1275052503
...,...,...,...,...,...,...,...
22696,93,2018-08-31,35.600,48.2400,35.1913,42.79,26382167
22697,93,2018-07-31,36.000,42.6300,33.9000,35.31,21627971
22698,93,2018-06-29,26.360,43.9799,26.3600,35.75,51552171
22699,93,2018-05-31,29.800,30.1000,24.7600,26.26,12958084


In [82]:
# Rename columns
renamed_trade_df = reduced_trade_df.rename(columns={"timestamp": "date", "ID": "ticker_id"})
renamed_trade_df

,ticker_id,date,open,high,low,close,volume
0,0,2023-09-12,189.485,189.9800,173.5400,176.30,500216459
1,0,2023-08-31,196.235,196.7300,171.9600,187.87,1323817340
2,0,2023-07-31,193.780,198.2300,186.6000,196.45,996368613
3,0,2023-06-30,177.700,194.4800,176.9306,193.97,1297863403
4,0,2023-05-31,169.280,179.3500,164.3100,177.25,1275052503
...,...,...,...,...,...,...,...
22696,93,2018-08-31,35.600,48.2400,35.1913,42.79,26382167
22697,93,2018-07-31,36.000,42.6300,33.9000,35.31,21627971
22698,93,2018-06-29,26.360,43.9799,26.3600,35.75,51552171
22699,93,2018-05-31,29.800,30.1000,24.7600,26.26,12958084


In [83]:
# Create a unique ID for each record
renamed_trade_df["id"] = range(len(renamed_trade_df))
id_column = renamed_trade_df.pop("id")
renamed_trade_df.insert(0, "id", id_column)

renamed_trade_df.head()

,id,ticker_id,date,open,high,low,close,volume
0,0,0,2023-09-12,189.485,189.98,173.5400,176.30,500216459
1,1,0,2023-08-31,196.235,196.73,171.9600,187.87,1323817340
2,2,0,2023-07-31,193.780,198.23,186.6000,196.45,996368613
3,3,0,2023-06-30,177.700,194.48,176.9306,193.97,1297863403
4,4,0,2023-05-31,169.280,179.35,164.3100,177.25,1275052503


In [84]:
# Export final DataFrame to csv file (for later import into DB)
clean_trade_data.to_csv("clean_db_data/clean_trade_data.csv", encoding='UTF-8', index=False)

## Create Metadata file

In [63]:
# Edit original DataFrame to only include columns of interest
reduced_overview = overview_4[["ID", "sector_id", "industry_id", "asset_type_id", "country_id", "currency_id",
                               "Symbol", "Name", "Description", "MarketCapitalization", 
                                "EBITDA", "BookValue", "DividendPerShare", "DividendYield"]]

In [64]:
# Rename columns
metadata = reduced_overview.rename(columns={"ID": "ticker_id", "Symbol": "symbol", "AssetType": "asset_type",
                                            "Name": "name", "Description": "description", "Currency": "currency",
                                            "Country": "country", "Sector": "sector", "Industry": "industry",
                                            "MarketCapitalization": "market_capitalization", "EBITDA": "ebitda",
                                            "BookValue": "book_value", "DividendPerShare": "dividend_per_share",
                                            "DividendYield": "dividend_yield"})
# Preview final DataFrame
metadata.head()

,ticker_id,sector_id,industry_id,asset_type_id,country_id,currency_id,symbol,name,description,market_capitalization,ebitda,book_value,dividend_per_share,dividend_yield
0,0,0,0,0,0,0,AAPL,Apple Inc,Apple Inc. is an American multinational techno...,2804149912000,123957002000,3.852,0.93,0.0054
1,1,0,1,0,0,0,MSFT,Microsoft Corporation,Microsoft Corporation is an American multinati...,2510813135000,102022996000,27.750,2.72,0.0080
2,2,1,2,0,0,0,AMZN,Amazon.com Inc,"Amazon.com, Inc. is an American multinational ...",1476477321000,63431000000,16.350,0.00,0.0000
3,3,2,3,0,0,0,NVDA,NVIDIA Corporation,Nvidia Corporation is an American multinationa...,1115896676000,12379000000,11.050,0.16,0.0004
4,4,0,4,0,0,0,META,Meta Platforms Inc.,"Meta Platforms, Inc. develops products that en...",793474040000,44914000000,51.590,0.00,0.0000


In [66]:
# Export final DataFrame to csv (for later import into DB)
metadata.to_csv("clean_db_data/clean_metadata.csv", encoding='UTF-8', index=False)